# Lab 1: Text Preprocessing, Indexing, and Compression

### Objective
To perform text preprocessing on a small document collection, analyze its effect on vocabulary size and term frequencies, construct inverted and positional indexes, and apply compression techniques on posting lists.


## Document Collection

We consider a collection of three short documents:

- **D1**: "Information retrieval is the process of obtaining relevant information."
- **D2**: "Retrieval systems use indexing and ranking techniques."
- **D3**: "Information systems retrieve data efficiently."

Each document is assigned a unique document ID.


In [2]:
documents = {
    1: "Information retrieval is the process of obtaining relevant information",
    2: "Retrieval systems use indexing and ranking techniques",
    3: "Information systems retrieve data efficiently"
}

## Text Preprocessing

The preprocessing pipeline consists of:
1. Tokenization
2. Stopword removal
3. Stemming
4. Lemmatization

Vocabulary size is defined as the number of unique terms in the document collection.


In [ ]:
#Required librarary components for text processing
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter

#Downlaod necessary NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    tokens = text.lower().split()
    tokens = [t for t in tokens if t not in stop_words]
    stemmed = [stemmer.stem(t) for t in tokens]
    lemmatized = [lemmatizer.lemmatize(t) for t in tokens]
    return tokens, stemmed, lemmatized


In [4]:
import pandas as pd
import re

# Helper function for raw tokenization (no preprocessing)
def raw_tokenize(text):
    return re.findall(r'\b\w+\b', text.lower())

original_tokens = []
processed_tokens = []
rows = []
processed_docs = {}

# Process each document
for doc_id, doc in documents.items():
    # Raw tokens
    raw = raw_tokenize(doc)
    original_tokens.extend(raw)

    # Preprocessed tokens (lemmatized)
    tokens, _, lemmas = preprocess(doc)
    processed_tokens.extend(lemmas)
    processed_docs[doc_id] = lemmas

    # Store row for comparison table
    rows.append([
        doc_id,
        doc,
        lemmas
    ])

# Display document-wise comparison
df = pd.DataFrame(
    rows,
    columns=["Doc", "Original", "Processed (Lemmatized)"]
)

print(df.to_string(index=False))

# Vocabulary statistics
original_vocab = set(original_tokens)
processed_vocab = set(processed_tokens)

original_tf = Counter(original_tokens)
processed_tf = Counter(processed_tokens)

reduction = (1 - len(processed_vocab) / len(original_vocab)) * 100


print("\n--- Analysis ---")
print(f"Original Vocabulary Size: {len(original_vocab)}")
print(f"Preprocessed Vocabulary Size: {len(processed_vocab)}")
print(f"Reduction: {reduction:.2f}%")

print("\nTop 5 Terms (Raw):", original_tf.most_common(5))
print("Top 5 Terms (Processed):", processed_tf.most_common(5))

 Doc                                                               Original                                              Processed (Lemmatized)
   1 Information retrieval is the process of obtaining relevant information [information, retrieval, process, obtaining, relevant, information]
   2                  Retrieval systems use indexing and ranking techniques              [retrieval, system, use, indexing, ranking, technique]
   3                          Information systems retrieve data efficiently                  [information, system, retrieve, data, efficiently]

--- Analysis ---
Original Vocabulary Size: 17
Preprocessed Vocabulary Size: 13
Reduction: 23.53%

Top 5 Terms (Raw): [('information', 3), ('retrieval', 2), ('systems', 2), ('is', 1), ('the', 1)]
Top 5 Terms (Processed): [('information', 3), ('retrieval', 2), ('system', 2), ('process', 1), ('obtaining', 1)]


## Inverted Index Construction

- An inverted index maps terms to the documents in which they occur.
- A positional index additionally stores the position of each term in a document.


In [5]:
from collections import defaultdict

class InvertedIndex:
    def __init__(self):
        # Basic inverted index: term -> list of document IDs
        self.index = defaultdict(list)

        # Positional inverted index: term -> {doc_id: [pos1, pos2, ...]}
        self.positional_index = defaultdict(lambda: defaultdict(list))

    def build(self, docs):
        for doc_id, tokens in docs.items():
            for pos, term in enumerate(tokens):
                # Basic Inverted Index (Avoid duplicate document IDs in posting list)
                if doc_id not in self.index[term]:
                    self.index[term].append(doc_id)

                # Positional Inverted Index (Store positions of terms in documents)
                self.positional_index[term][doc_id].append(pos)

    def get_postings(self, term):
        return self.index.get(term, [])

    def get_positional_postings(self, term):
        return self.positional_index.get(term, {})
    
# Build the index
ir_system = InvertedIndex()
ir_system.build(processed_docs)

# Display the inverted index & positional index
print(f"{'Term':<15} | {'Postings List (DocID)':<20} | {'Positional Index (DocID: [Positions])'}")
print("-" * 90)
# Sort terms alphabetically for clean output
sorted_terms = sorted(ir_system.index.keys())

for term in sorted_terms:
    postings = ir_system.index[term]
    positional = dict(ir_system.positional_index[term])
    print(f"{term:<15} | {str(postings):<20} | {positional}")


Term            | Postings List (DocID) | Positional Index (DocID: [Positions])
------------------------------------------------------------------------------------------
data            | [3]                  | {3: [3]}
efficiently     | [3]                  | {3: [4]}
indexing        | [2]                  | {2: [3]}
information     | [1, 3]               | {1: [0, 5], 3: [0]}
obtaining       | [1]                  | {1: [3]}
process         | [1]                  | {1: [2]}
ranking         | [2]                  | {2: [4]}
relevant        | [1]                  | {1: [4]}
retrieval       | [1, 2]               | {1: [1], 2: [0]}
retrieve        | [3]                  | {3: [2]}
system          | [2, 3]               | {2: [1], 3: [1]}
technique       | [2]                  | {2: [5]}
use             | [2]                  | {2: [2]}


## Query Processing

- **Single-term query**: Find docs containing "information".
- **Boolean query**: "(data AND retrieve)NOT indexing", "information OR use".
- **Phrase query**: "retrieve data". Requires checking if positions are adjacent


In [6]:
def print_postings(term):
    postings = set(ir_system.get_postings(term))
    print(f"Postings({term}) = {sorted(postings)}")
    return postings


def single_term_query(term):
    print(f"\nSINGLE-TERM QUERY: '{term}'")
    print("-" * 50)
    return print_postings(term)


def boolean_query(query_name, include_terms, exclude_terms=None):
    print(f"\nBOOLEAN QUERY: {query_name}")
    print("-" * 50)

    result = None

    # AND operation for included terms
    for term in include_terms:
        postings = print_postings(term)
        result = postings if result is None else result & postings

    # NOT operation for excluded terms
    if exclude_terms:
        for term in exclude_terms:
            exclude_postings = print_postings(term)
            result -= exclude_postings

    print(f"Final Result = {sorted(result)}")
    return sorted(result)


def phrase_query(term1, term2):
    print(f'\nPHRASE QUERY: "{term1} {term2}"')
    print("-" * 50)

    pos1 = ir_system.get_positional_postings(term1)
    pos2 = ir_system.get_positional_postings(term2)

    result_docs = []

    for doc_id in pos1.keys() & pos2.keys():
        print(f"\nDocument {doc_id}:")
        print(f"  {term1} positions: {pos1[doc_id]}")
        print(f"  {term2} positions: {pos2[doc_id]}")

        if any(p + 1 in pos2[doc_id] for p in pos1[doc_id]):
            print("  → Phrase match found")
            result_docs.append(doc_id)

    if not result_docs:
        print("\nNo phrase match found.")

    return result_docs

# Single-term
single_term_query("information")

# Boolean
boolean_query(
    "(data AND retrieve) NOT indexing",
    include_terms=["data", "retrieve"],
    exclude_terms=["indexing"]
)

boolean_query(
    "information OR use",
    include_terms=["information", "use"]
)

# Phrase
phrase_query("retrieve", "data")



SINGLE-TERM QUERY: 'information'
--------------------------------------------------
Postings(information) = [1, 3]

BOOLEAN QUERY: (data AND retrieve) NOT indexing
--------------------------------------------------
Postings(data) = [3]
Postings(retrieve) = [3]
Postings(indexing) = [2]
Final Result = [3]

BOOLEAN QUERY: information OR use
--------------------------------------------------
Postings(information) = [1, 3]
Postings(use) = [2]
Final Result = []

PHRASE QUERY: "retrieve data"
--------------------------------------------------

Document 3:
  retrieve positions: [2]
  data positions: [3]
  → Phrase match found


[3]

## Index Compression

Posting lists are converted to gap representation and then encoded to reduce space.




### 1. Gap (Delta) Encoding

Instead of storing absolute document IDs in posting lists, we store the difference
(gap) between consecutive document IDs.

Given a sorted posting list:
d₁, d₂, d₃, ..., dₙ

Gap representation is computed as:
gap₁ = d₁  
gapᵢ = dᵢ − dᵢ₋₁  for i > 1

Since gaps are usually small integers, they are more efficient to encode.


In [24]:
term = "information"
posting_list = [1, 3]  

print(f"Term: {term}")
print(f"Posting List: {posting_list}")

print("\n=== ORIGINAL REPRESENTATION ===")

original_bits = len(posting_list) * 32
print(f"Assumption: 32 bits per DocID")
print(f"Total Bits Used: {original_bits}")

Term: information
Posting List: [1, 3]

=== ORIGINAL REPRESENTATION ===
Assumption: 32 bits per DocID
Total Bits Used: 64


In [25]:
def compute_gaps(posting_list):
    if not posting_list:
        return []
    gaps = [posting_list[0]]
    for i in range(1, len(posting_list)):
        gaps.append(posting_list[i] - posting_list[i - 1])
    return gaps


gaps = compute_gaps(posting_list)

print("\n=== GAP ENCODING ===")
print(f"Gaps: {gaps}")

gap_bits = sum(g.bit_length() for g in gaps)
print(f"Bits Used (minimal binary representation): {gap_bits}")



=== GAP ENCODING ===
Gaps: [1, 2]
Bits Used (minimal binary representation): 3


### 2. Variable-Byte Encoding

Variable-Byte encoding represents integers using a variable number of bytes.

Each byte:
- Uses the most significant bit (MSB) as a continuation flag
- MSB = 1 → last byte of the number
- MSB = 0 → more bytes follow
- Remaining 7 bits store the actual data
- Example: 5 (binary 101) fits in 7 bits. Encoded: 10000101 (Hex 85).

In [26]:
#Encodes a single integer using VB encoding
def vb_encode_number(number):
    bytes_list = []
    while True:
        byte = number % 128
        if not bytes_list:
            # MSB = 1 indicates last byte
            byte += 128
        bytes_list.insert(0, byte)
        number //= 128
        if number == 0:
            break
    return bytes_list


def vb_encode(gaps):
    stream = []
    for gap in gaps:
        stream.extend(vb_encode_number(gap))
    return stream


vb_encoded = vb_encode(gaps)

print("\n=== VARIABLE BYTE ENCODING ===")
print("Encoded Bytes (Decimal):", vb_encoded)
print("Encoded Bytes (Hex):", [hex(b) for b in vb_encoded])
print("Encoded Bytes (Binary):", [format(b, '08b') for b in vb_encoded])

vb_bits = len(vb_encoded) * 8
print(f"Total Bits Used: {vb_bits}")



=== VARIABLE BYTE ENCODING ===
Encoded Bytes (Decimal): [129, 130]
Encoded Bytes (Hex): ['0x81', '0x82']
Encoded Bytes (Binary): ['10000001', '10000010']
Total Bits Used: 16


### 3. Golomb Encoding

Golomb encoding is a lossless compression technique suitable for sequences of
geometrically distributed integers, such as gap values in posting lists.

For a number x and parameter M:
- Quotient q = ⌊x / M⌋ is encoded using Unary coding
- Remainder r = x mod M is encoded using Binary coding

Golomb encoding is particularly effective when the average gap is small.


In [27]:
import math

def golomb_encode(gaps, M):
    b = math.ceil(math.log2(M))
    cutoff = 2**b - M
    bit_stream = ""

    for x in gaps:
        q = x // M
        r = x % M

        # Unary encoding of quotient
        bit_stream += "1" * q + "0"

        # Binary encoding of remainder
        if r < cutoff:
            bit_stream += format(r, f"0{b-1}b")
        else:
            bit_stream += format(r + cutoff, f"0{b}b")

    return bit_stream


mean_gap = sum(gaps) / len(gaps)
M = max(1, int(0.69 * mean_gap))

golomb_bits = golomb_encode(gaps, M)

print("\n=== GOLOMB ENCODING ===")
print(f"Chosen M: {M}")
print("Encoded Bit Stream:", golomb_bits)
print(f"Total Bits Used: {len(golomb_bits)}")



=== GOLOMB ENCODING ===
Chosen M: 1
Encoded Bit Stream: 1001100
Total Bits Used: 7


In [28]:
print("\n=== COMPRESSION COMPARISON SUMMARY ===")
print("-" * 40)
print(f"Original (32-bit integers): {original_bits} bits")
print(f"Gap Encoding Only:          {gap_bits} bits")
print(f"Variable Byte Encoding:     {vb_bits} bits")
print(f"Golomb Encoding:            {len(golomb_bits)} bits")



=== COMPRESSION COMPARISON SUMMARY ===
----------------------------------------
Original (32-bit integers): 64 bits
Gap Encoding Only:          3 bits
Variable Byte Encoding:     16 bits
Golomb Encoding:            7 bits
